# Vector Store

This notebook demonstrates how to work with vector stores for semantic search and retrieval.

We'll cover:
- Loading and chunking text documents
- Storing document embeddings in different vector databases
- Querying and retrieving relevant documents
- Using vector stores as configurable LangChain runnables

In [1]:
from dotenv import load_dotenv
from rich import print

load_dotenv(verbose=True)

# %load_ext autoreload
# %autoreload 2

# import sys

# sys.path.append(".")

True

### Split the text into chunks

First we load a text document and split it into smaller chunks for processing:

- Uses LangChain's `TextLoader` to load the file
- Applies `RecursiveCharacterTextSplitter` to break text into 2000-character chunks
- No overlap between chunks is configured

In [2]:
from genai_tk.core.embeddings_factory import EmbeddingsFactory
from genai_tk.core.embeddings_store import VECTOR_STORE_ENGINE, EmbeddingsStore
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader("use_case_data/other/state_of_the_union.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
print(texts)

[
    Document(
        metadata={'source': 'use_case_data/other/state_of_the_union.txt'},
        page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress
and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This 
year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most 
importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with 
an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin 
sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly
miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of 
strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their
fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with 
their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland. \n\nIn this 
struggle as President Zelenskyy said in his speech to the European Parliament “Light will win over darkness.” The 
Ukrainian Ambassador to the United States is here tonight. \n\nLet each of us here tonight in this Chamber send an 
unmistakable signal to Ukraine and to the world. \n\nPlease rise if you are able and show that, Yes, we the United 
States of America stand with the Ukrainian people. \n\nThroughout our history we’ve learned this lesson when 
dictators do not pay a price for their aggression they cause more chaos.   \n\nThey keep moving.   \n\nAnd the 
costs and the threats to America and the world keep rising.   \n\nThat’s why the NATO Alliance was created to 
secure peace and stability in Europe after World War 2. \n\nThe United States is a member along with 29 other 
nations. \n\nIt matters. American diplomacy matters. American resolve matters.'
    ),
    Document(
        metadata={'source': 'use_case_data/other/state_of_the_union.txt'},
        page_content='Putin’s latest attack on Ukraine was premeditated and unprovoked. \n\nHe rejected repeated 
efforts at diplomacy. \n\nHe thought the West and NATO wouldn’t respond. And he thought he could divide us at home.
Putin was wrong. We were ready.  Here is what we did.   \n\nWe prepared extensively and carefully. \n\nWe spent 
months building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to 
confront Putin. \n\nI spent countless hours unifying our European allies. We shared with the world in advance what 
we knew Putin was planning and precisely how he would try to falsely justify his aggression.  \n\nWe countered 
Russia’s lies with truth.   \n\nAnd now that he has acted the free world is holding him accountable. \n\nAlong with
twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United 
Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. \n\nWe are inflicting 
pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 
\n\nTogether with our allies –we are right now enforcing powerful economic sanctions. \n\nWe are cutting off 
Russia’s largest banks from the international financial system.  \n\nPreventing Russia’s central bank from 
defending the Russian Ruble making Putin’s $630 Billion “war fund” worthless.   \n\nWe are choking off Russia’s 
access to technology that will sap its economic strength and weaken its military for years to come.  \n\nTonight I 
say to the Russian oligarchs and corrupt leaders who have bilked billions of dollars off this violent regime no 
more. \n\nThe U.S. Department of Justice is assembling a dedicated task force to go after the crimes of Russ

### Store document embeddings in a vector database

We use our `EmbeddingsStore` to:

1. Select a vector store backend (default is in-memory)
2. Configure the embedding model (default from config)
3. Add our document chunks to the store

Key benefits of the factory pattern:
- Easy switching between vector store implementations
- Consistent interface regardless of backend
- Centralized configuration management

In [3]:
embeddings_store = EmbeddingsStore.create_from_config("in_memory")

print(embeddings_store)

db = embeddings_store.get()
db.add_documents(texts)

EmbeddingsStore(
    backend='InMemory',
    embeddings_factory=EmbeddingsFactory(
        embeddings='default',
        embeddings_id='qwen3_06b_deepinfra',
        embeddings_tag=None,
        encoding_str=None,
        retrieving_str=None,
        cache_embeddings=False,
        info=EmbeddingsInfo(
            id='qwen3_06b_deepinfra',
            provider='deepinfra',
            model='Qwen/Qwen3-Embedding-0.6B',
            prefix='',
            dimension=1024
        )
    ),
    table_name_prefix='embeddings',
    config={},
    index_document=False,
    collection_metadata=None,
    record_manager_url=None,
    table_name='embeddings_qwen3_06b',
    description='InMemory/embeddings_qwen3_06b'
)

/tmp/ipykernel_31498/2270734943.py:5: DeprecationWarning: use get_vector_store
  db = embeddings_store.get()
2026-02-03 14:26:01.127 | DEBUG    | genai_tk.core.embeddings_store:get_vector_store:428 - get vector store  : InMemory/embeddings_qwen3_06b


['7c29b305-1c34-4788-85fc-bd7d8a48c299',
 '729fefeb-6439-4c43-a759-622e8e4e03fd',
 '80613923-4f5e-46c0-aa95-8feed3c9d333',
 'a4ff6945-06b6-46ec-87f0-49f8ff5f2d82',
 '385078f9-fbe7-45c5-a0fb-c0f5ced28712',
 '6036cdb0-63f8-40f0-b1f4-540ff753e307',
 'de26b62a-f1b9-445d-9600-768ffa32dccd',
 'db4042b2-a7cb-4370-a13d-6fb093b67bfe',
 '21daba96-edbc-4f91-b1d2-c1eec738091f',
 '0f3fe55a-8139-4ffe-bc85-7b2342adcb67',
 '1a1d1347-84c4-40ef-b111-6adc179ab949',
 '5651b05e-68db-4a69-a181-c0c56c5230c8',
 'eac6d557-0b86-490c-81c6-2636c5d3acc7',
 '9bbd515b-4846-4de1-a317-29332eaeb66d',
 'bb5278f7-ac03-4871-85ca-6b492fa7811a',
 'd5f9fbfd-bee8-44f5-b320-c3d9bab8d00f',
 '51ff74d7-9363-422c-ba1a-b14b1cae7477',
 'b19f6620-2e8b-4991-a754-98682f74a936',
 '79cf9b58-e291-45c8-a515-d4e1169e4b25',
 'accd506a-b886-4765-aa83-aea1b8f7793b',
 '04cd92ad-b6e8-4500-acdd-92f563a37b2f']

### Test semantic search queries

We'll search for content related to:
1. "What did the president say about Ketanji Brown Jackson" (English)
2. "Qu'as dit le président sur Ketanji Brown Jackson" (French)

This demonstrates:
- The vector store finds relevant content regardless of query language
- Semantic similarity works across languages when using multilingual embeddings

In [4]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query, k=3)
print(docs)

[
    Document(
        id='d5f9fbfd-bee8-44f5-b320-c3d9bab8d00f',
        metadata={'source': 'use_case_data/other/state_of_the_union.txt'},
        page_content='In state after state, new laws have been passed, not only to suppress the vote, but to 
subvert entire elections. \n\nWe cannot let this happen. \n\nTonight. I call on the Senate to: Pass the Freedom to 
Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can 
know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this 
country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States 
Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional 
responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did 
that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top 
legal minds, who will continue Justice Breyer’s legacy of excellence. \n\nA former top litigator in private 
practice. A former federal public defender. And from a family of public school educators and police officers. A 
consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of 
Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, 
we need to secure the Border and fix the immigration system. \n\nWe can do both. At our border, we’ve installed new
technology like cutting-edge scanners to better detect drug smuggling.  \n\nWe’ve set up joint patrols with Mexico 
and Guatemala to catch more human traffickers.  \n\nWe’re putting in place dedicated immigration judges so families
fleeing persecution and violence can have their cases heard faster. \n\nWe’re securing commitments and supporting 
partners in South and Central America to host more refugees and secure their own borders.'
    ),
    Document(
        id='7c29b305-1c34-4788-85fc-bd7d8a48c299',
        metadata={'source': 'use_case_data/other/state_of_the_union.txt'},
        page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress
and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This 
year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most 
importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with 
an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin 
sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly
miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of 
strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their
fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with 
their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland. \n\nIn this 
struggle as President Zelenskyy said in his speech to the European Parliament “Light will win over darkness.” The 
Ukrainian Ambassador to the United States is here tonight. \n\nLet each of us here tonight in this Chamber send an 
unmistakable signal to Ukraine and to the world. \n\nPlease rise if you are able and show that, Yes, we the United 
States of America stand with the Ukrainian people. \n\nThroughout our history we’ve learned this lesson when 
dictators do not pay a price for their aggression they cause more chaos.   \n\nThey keep moving.   \n\nAnd the 
costs and the threats to America and the world keep rising.   \n\nThat’s why the NATO Allianc

In [5]:
query = "Qu'as dit le président sur Ketanji Brown Jackson"
docs = db.similarity_search(query, k=3)
print(docs)

[
    Document(
        id='d5f9fbfd-bee8-44f5-b320-c3d9bab8d00f',
        metadata={'source': 'use_case_data/other/state_of_the_union.txt'},
        page_content='In state after state, new laws have been passed, not only to suppress the vote, but to 
subvert entire elections. \n\nWe cannot let this happen. \n\nTonight. I call on the Senate to: Pass the Freedom to 
Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can 
know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this 
country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States 
Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional 
responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did 
that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top 
legal minds, who will continue Justice Breyer’s legacy of excellence. \n\nA former top litigator in private 
practice. A former federal public defender. And from a family of public school educators and police officers. A 
consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of 
Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, 
we need to secure the Border and fix the immigration system. \n\nWe can do both. At our border, we’ve installed new
technology like cutting-edge scanners to better detect drug smuggling.  \n\nWe’ve set up joint patrols with Mexico 
and Guatemala to catch more human traffickers.  \n\nWe’re putting in place dedicated immigration judges so families
fleeing persecution and violence can have their cases heard faster. \n\nWe’re securing commitments and supporting 
partners in South and Central America to host more refugees and secure their own borders.'
    ),
    Document(
        id='7c29b305-1c34-4788-85fc-bd7d8a48c299',
        metadata={'source': 'use_case_data/other/state_of_the_union.txt'},
        page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress
and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This 
year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most 
importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with 
an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin 
sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly
miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of 
strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their
fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with 
their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland. \n\nIn this 
struggle as President Zelenskyy said in his speech to the European Parliament “Light will win over darkness.” The 
Ukrainian Ambassador to the United States is here tonight. \n\nLet each of us here tonight in this Chamber send an 
unmistakable signal to Ukraine and to the world. \n\nPlease rise if you are able and show that, Yes, we the United 
States of America stand with the Ukrainian people. \n\nThroughout our history we’ve learned this lesson when 
dictators do not pay a price for their aggression they cause more chaos.   \n\nThey keep moving.   \n\nAnd the 
costs and the threats to America and the world keep rising.   \n\nThat’s why the NATO Allianc

### Vector Store as Runnable

LangChain's `as_retriever()` converts the vector store into a runnable component that can:

- Be chained with other LangChain components
- Support streaming and async operations
- Be configured with search parameters

In [6]:
retriever = db.as_retriever()

a = retriever.invoke(query, k=1)
print(a)

[
    Document(
        id='d5f9fbfd-bee8-44f5-b320-c3d9bab8d00f',
        metadata={'source': 'use_case_data/other/state_of_the_union.txt'},
        page_content='In state after state, new laws have been passed, not only to suppress the vote, but to 
subvert entire elections. \n\nWe cannot let this happen. \n\nTonight. I call on the Senate to: Pass the Freedom to 
Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can 
know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this 
country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States 
Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional 
responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did 
that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top 
legal minds, who will continue Justice Breyer’s legacy of excellence. \n\nA former top litigator in private 
practice. A former federal public defender. And from a family of public school educators and police officers. A 
consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of 
Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, 
we need to secure the Border and fix the immigration system. \n\nWe can do both. At our border, we’ve installed new
technology like cutting-edge scanners to better detect drug smuggling.  \n\nWe’ve set up joint patrols with Mexico 
and Guatemala to catch more human traffickers.  \n\nWe’re putting in place dedicated immigration judges so families
fleeing persecution and violence can have their cases heard faster. \n\nWe’re securing commitments and supporting 
partners in South and Central America to host more refugees and secure their own borders.'
    )
]